**Task_no_01**

In [12]:
import glob as glob
import os
import pandas as pd
import shutil


files = glob.glob("/content/sample_data/*.csv")
for file in files:
  shutil.move(file, "/content/backup_folder/")
  print(f"Moved file: {file}")

def export_data(df, filename, format):
  if format == "csv":
    df.to_csv(filename, index=False)
    print(f"Data exported to {filename} in CSV format.")
  elif format == "json":
    df.to_json(filename, orient="records")
    print(f"Data exported to {filename} in JSON format.")
  else:
    print("Unsupported format.")
# Example usage:
# Creating a sample dataframe
data = {'Name': ['Alice', 'Bob', 'Charlie'],
'Age': [25, 30, 35],
'City': ['New York', 'Los Angeles', 'Chicago']}
df = pd.DataFrame(data)
# Exporting to CSV
export_data(df, "output.csv", "csv")
# Exporting to JSON
export_data(df, "output.json", "json")

Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


**Task_no_2**

In [21]:
import time
import sqlite3
import yfinance as yf
import pandas as pd

# Establish a connection to SQLite database
conn = sqlite3.connect("stocks.db")  # Creates a database file
cursor = conn.cursor()

# Create a table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)''')
conn.commit()  # Commit the table creation

# Function to fetch stock data
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None  # Return None if no data is available

        latest = data.iloc[-1]  # Get the most recent price data

        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to store data in SQLite
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)

    if stock_data:  # Only store if data is available
        cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                          VALUES (?, ?, ?, ?, ?, ?)''',
                       (stock_data["symbol"], stock_data["open"],
                        stock_data["high"], stock_data["low"],
                        stock_data["close"], stock_data["volume"]))
        conn.commit()
        print(f"Stored data for {symbol}")

# Function to analyze stock data
def analyze_stock(symbol):
    df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100",
                           conn, params=(symbol,))
    print(df)

# Example Usage
symbol = "AAPL"  # Apple stock
for _ in range(5):  # Fetch data 5 times with intervals
    store_data(symbol)

# Analyze stored stock data
analyze_stock(symbol)

# Close database connection
conn.close()


Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AAPL: Too Many Requests. Rate limited. Try after a while.
Empty DataFrame
Columns: [id, symbol, timestamp, open, high, low, close, volume]
Index: []


**Task_no_03**

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
URL = "http://books.toscrape.com/"
HEADERS = {"User-Agent": "Mozilla/5.0"}
def get_books(url):
  response = requests.get(url, headers=HEADERS)
  soup = BeautifulSoup(response.text, "html.parser")
  books = soup.find_all("article", class_="product_pod")
  book_list = []
  for book in books:
    title = book.h3.a["title"]
    price = book.find("p", class_="price_color").text
    stock = book.find("p", class_="instock availability").text.strip()
    book_list.append({"Title": title, "Price": price, "Availability":
    stock})
  return book_list
books_data = get_books(URL)
df = pd.DataFrame(books_data)
df.to_csv("books.csv", index=False)
print("Data saved to books.csv")

Data saved to books.csv
